In [ ]:
import numpy as np

# create class to keep track of state
class Board:
    def __init__(self, start_x, start_y, dest_x, dest_y, size):
        if start_x > size-1 or start_y > size-1 or dest_x > size-1 or dest_y > size-1:
            raise Exception("Placement coordinates cannot be greater than the size of the board")
        if start_x == dest_x and start_y == dest_y:
            raise Exception("Start place and destination cannot be the same")
        fill_value = " "
        # create initial board 
        self.size = size
        self.board = np.array([[fill_value for _ in range(size)] for _ in range(size)])
        self.current_x, self.current_y = start_x, start_y
        self.dest_x, self.dest_y = dest_x, dest_y
        self.arrived = False

        # add initial coordinates
        self.board[start_x, start_y] = "o"
        self.board[dest_x, dest_y] = "x"

    def move(self, direction):
        if direction not in ["up", "down", "left", "right"]:
            raise Exception("Invalid direction")
        
        next_x, next_y = self.current_x, self.current_y
        match direction:
            case "up":
                if self.current_y > 0:
                    next_y = self.current_y-1
            case "down":
                if self.current_y < self.size - 1:
                    next_y = self.current_y+1
            case "left":
                if self.current_x > 0:
                    next_x = self.current_x-1
            case "right":
                if self.current_x < self.size-1:
                    next_x = self.current_x+1

        # swap to new location
        temp = self.board[next_y, next_x]
        self.board[next_y, next_x] = self.board[self.current_y, self.current_x]
        self.board[self.current_y, self.current_x] = temp
        # set new currents
        self.current_x, self.current_y = next_x, next_y
        if next_x == self.dest_x and next_y == self.dest_y:
            self.arrived = True
            print("You have arrived!")

    def hasArrived(self):
        return self.arrived

    def toString(self):
        return str(self.board)
        
    def print(self):
        print(self.board)

board = Board(0,0,6,9,10)
board.print()

In [ ]:
# create openai client
from openai import OpenAI

key = open("key.txt").read()
client = OpenAI(api_key=key)

In [ ]:
import json

tools = [{
    "type": "function",
    "function": {
        "name": "move",
        "description": "moves the users location on the board",
        "parameters": {
            "type": "object",
            "properties": {
                "direction": {
                    "type": "string",
                    "enum": ["up", "down", "left", "right"]
                }
            }
        },
        "required": ["direction"]
    }
}]

board = Board(0,0, 5,7, 10)

messages = [
        {
            "role": "system",
            "content": "You are given a board with your current location marked as `o`. You must move you're position to `x` in the most efficient way using the function calls provided.",
        },
        {
            "role": "user",
            "content": board.toString(),
        },
]

board.print()

while (not board.hasArrived()):
    chat_completion = client.chat.completions.create(
        messages=messages,
        model="gpt-3.5-turbo",
        tools=tools,
        tool_choice="auto"
    )

    content = "You made an incorrect function call."

    try:
        print(chat_completion.choices[0].message.tool_calls[0].function)
        args = json.loads(chat_completion.choices[0].message.tool_calls[0].function.arguments)
        board.move(args["direction"])
        content = f"You moved {args['direction']}"
    except Exception as e:
        print(e)

    board.print()
    messages.append({"role": "user", "content": content + "\n" + board.toString()})